# 机器学习与社会科学应用

# 第四章 自然语言处理入门

# 第三节 TF-IDF

<font face="宋体" >郭峰    
    教授、博士生导师  
上海财经大学公共经济与管理学院  
上海财经大学数实融合与智能治理实验室    
    邮箱：guofengsfi@163.com</font> 

In [ ]:
# 目标：用语料库表征两个文本，然后计算他们的相似度
# 这个例子没有考虑到自定义词典和停用词

In [ ]:
# 安装gensim: pip install gensim
import gensim 
print(gensim.__version__)

In [ ]:
import jieba
from gensim import corpora,models,similarities

# 语料文档
doc0 = "我不喜欢上海"
doc1 = "上海是一个好地方"
doc2 = "北京是一个好地方"
doc3 = "上海好吃的在哪里"
doc4 = "上海好玩的在哪里"
doc5 = "上海是好地方"
doc6 = "上海路和上海人"
doc7 = "喜欢小吃"

# 测试文档
doc_test1 = "我喜欢上海的小吃"
doc_test2 = "我不喜欢上海"

In [ ]:
# 分词
# 为了简化操作，把目标文档放到一个列表all_doc
all_doc = []
all_doc.append(doc0)
all_doc.append(doc1)
all_doc.append(doc2)
all_doc.append(doc3)
all_doc.append(doc4)
all_doc.append(doc5)
all_doc.append(doc6)
all_doc.append(doc7)
print(all_doc)


In [ ]:
# 对目标文档进行分词，并且保存在列表all_doc_list中
all_doc_list = []
for doc in all_doc:
    doc_list = [word for word in jieba.cut(doc)]
    all_doc_list.append(doc_list)

all_doc_list

In [ ]:
# 制作语料库，首先用dictionary方法获取词袋（bag-of-words)
dictionary = corpora.Dictionary(all_doc_list)
print(len(dictionary))
print(dictionary.keys())
print(list(dictionary.values()))
print(dictionary[0])
print(dictionary[1])
print(dictionary)

In [ ]:
# 编号与词之间的对应关系
dictionary.token2id

In [ ]:
# 使用doc2bow制作语料库
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]
corpus

In [ ]:
# 使用TF-IDF模型对语料库建模
tfidf = models.TfidfModel(corpus, dictionary=dictionary)

In [ ]:
# 测试文档也进行分词，并保存在列表doc_test_list中
doc_test1_cut = [word for word in jieba.cut(doc_test1)]
doc_test2_cut = [word for word in jieba.cut(doc_test2)]
print(doc_test1_cut)
print(doc_test2_cut)

In [ ]:
# 用同样的方法，把测试文档也转换为二元组的向量
doc_test1_vec = dictionary.doc2bow(doc_test1_cut)
doc_test2_vec = dictionary.doc2bow(doc_test2_cut)
print(doc_test1_vec)
print(doc_test2_vec)

In [ ]:
# 计算测试文档的tfidf向量表示
doc_test1_tfidf = tfidf[doc_test1_vec]
print(doc_test1_tfidf)
doc_test2_tfidf = tfidf[doc_test2_vec]
print(doc_test2_tfidf)

In [ ]:
# 两个测试文档相似度的计算
# gensim自带的similarities算法使用方法为计算某一文档（目标文档）与另外一组文档（对比文档集）的相似度，输出也是一个列表
# 先将目标文档集列表化，这里列表只有一个元素

doc_test1_list = [doc_test1_vec]
doc_test1_tfidf = tfidf[doc_test1_list]
print(doc_test1_tfidf[0])

In [ ]:
# 对文档集相似度计算进行建模
tfidf_sim = similarities.SparseMatrixSimilarity(doc_test1_tfidf,num_features=len(dictionary.keys()))

In [ ]:
# 计算目标文档与对比文档集（这里只有一个文件）的相似度列表
sim = tfidf_sim[doc_test2_tfidf]
print(sim[0])

In [ ]:
# 余弦相似度的具体计算（解析上面的相似度0.54680777到底是怎么得来的）
vec1 = [0.08112725037593049,0,0.3909393754390612,0.5864090631585919,0,0,0,0,0,0,0,0,0.3909393754390612,0,0,0,0,0.58640906315859,0]
vec2 = [0.08814189721744814,0.6371127720068723,0.4247418480045816,0.6371127720068723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
def similarity(a_vect, b_vect):
    dot_val = 0.0
    a_norm = 0.0
    b_norm = 0.0
    cos = None
    for a, b in zip(a_vect, b_vect):
        dot_val += a*b
        a_norm += a**2
        b_norm += b**2
    if a_norm == 0.0 or b_norm == 0.0:
        cos = -1
    else:
        cos = dot_val / ((a_norm*b_norm)**0.5)
    return cos
print(similarity(vec1,vec2))

In [ ]:
# 计算目标文档与测试集所有文档的相似度
print(corpus)
doc_train_tfidf=tfidf[corpus]
print(doc_train_tfidf)


In [ ]:
tfidf_sim = similarities.SparseMatrixSimilarity(doc_train_tfidf, num_features=len(dictionary.keys()))

In [ ]:
doc_test1_tfidf = tfidf[doc_test1_vec]
sim1 = tfidf_sim[doc_test1_tfidf]
print(sim1)
print(sim1.max())

In [ ]:
doc_test2_tfidf = tfidf[doc_test2_vec]
sim2 = tfidf_sim[doc_test2_tfidf]
print(sim2)
print(sim2.max())  # 注：目标文档2就是测试集中的一个文档，所以相似度为1

In [ ]:
# 输出与目标文档最最相似文档的编号（文档）及相似度数值
# 根据相似度排序
sim1_new = sorted(enumerate(sim1), key=lambda item: -item[1])
# 从分析结果来看，测试文档与doc7相似度最高，其次是doc0，与doc2的相似度为零
print(sim1_new)

In [ ]:
print(sim1_new[0][1])
print(all_doc)
print(all_doc[sim1_new[0][0]])

In [ ]:
# 本节结束